### EDA

In [3]:
library(cowplot) #Installing package into 'C:/Users/User/Documents/R/win-library/4.0'
library(grid)
library(ggplot2)
library(ggExtra)
library(tidyverse)
library(lubridate)
library(gridExtra)
library(tableone)

In [3]:
# install.packages("tableone")

In [4]:
# directories
datadir = "../../DataTD/shc2021"
outdir = "../../OutputTD/shc2021"
figuredir = "../../OutputTD/Figures"
tabledir = "../../OutputTD/4_tables"

options(repr.matrix.max.rows=350, repr.matrix.max.cols=30)

In [6]:
adt <- read.csv(file.path(datadir, "cohort3_adt_2021.csv"))
nrow(adt)
length(unique(adt$pat_enc_csn_id_coded))

[1] 1181264

[1] 61176

In [12]:
head(adt, 2)

,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,department_id
,<chr>,<dbl>,<chr>,<int>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>
1,JC2210295,1.31260e+11,2018-12-13 07:59:00+00:00,12,Inpatient,NA,5,Acute Care (Assessment or intervention q4-8),Census,Cystic Fibrosis Adult,2000238
2,JC1259166,1.31252e+11,2018-07-05 06:59:00+00:00,21,Inpatient,NA,9,IICU/Intermediate Care (Assessment or intervention q2-4),Census,Pulmonary Hypertension,2000237


In [9]:
demo_deaths <- read.csv(file.path(datadir, "cohort3_demo_deaths_2021.csv"))
nrow(demo_deaths)
length(unique(demo_deaths$anon_id))

[1] 41015

[1] 41015

In [10]:
head(demo_deaths)

,anon_id,death_date_jittered
,<chr>,<chr>
1,JC2886880,2021-02-26
2,JC1556282,
3,JC549302,
4,JC1633831,
5,JC758980,
6,JC1985456,


In [17]:
dx <- read.csv(file.path(datadir, "diagnosis_2021.csv")) # %>% mutate(admit_time = ymd_hms(admit_time))
nrow(dx) #3640536

dept <- read.csv(file.path(datadir, "dept_2021.csv")) 
nrow(dept) #1171180

[1] 3640536

[1] 1171180

[1] 2649157

In [18]:
head(dx, 1)
head(dept, 1)

,dx_name,icd10,pat_enc_csn_id_jittered
,<chr>,<chr>,<dbl>
1,"Hodgkin's disease, unspecified type, of lymph nodes of multiple sites",C81.98,131066472308


,dept_id,dept_name
,<int>,<chr>
1,110100017,M7


,anon_id,pat_enc_csn_id_coded,name,prov_map_id,treatment_start_time,admit_time
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,JC1000116,131066472308,Primary Team,SG1000039,2015-01-27 10:21:00+00:00,2015-01-28 00:46:00+00:00


### Department - look for OR

In [10]:
dept %>% group_by(dept_id, dept_name) %>% summarize(n=n()) %>% arrange(n)

`summarise()` has grouped output by 'dept_id'. You can override using the `.groups` argument.



dept_id,dept_name,n
<int>,<chr>,<int>
120201007,VCP PEDIATRICS,4
1107301002,CATH AMC,8
7101005,ASC SHORT STAY,11
110100410,ACA6 ICU,26
20026201,D2ICU-SURGE,27
7301007,ZZCATH ASC PRE-OP,30
120201002,VCP 2 NORTH,41
7301006,CAPR XFER OVERFL,44
120201021,VCP SURGERY,60


In [11]:
dept %>% group_by(dept_id, dept_name) %>% summarize(n=n()) %>% arrange(dept_id)

`summarise()` has grouped output by 'dept_id'. You can override using the `.groups` argument.



dept_id,dept_name,n
<int>,<chr>,<int>
2000211,B1,21379
2000222,G2P,27282
2000224,C2,28929
2000231,B3,50473
2000232,FGR,32691
2000233,H1,28485
2000237,B2,42311
2000238,F3,22031
2000241,D2,6996


In [12]:
dept %>% group_by(dept_id, dept_name) %>% summarize(n=n()) %>% arrange(dept_name)

`summarise()` has grouped output by 'dept_id'. You can override using the `.groups` argument.



dept_id,dept_name,n
<int>,<chr>,<int>
6001000,500P INTERVENTIONAL PLATFORM,140
6001002,500P INTRA-OP,10197
6001003,500P PACU,8648
6001001,500P PRE-OP,4958
110100310,ACA6 AAU,66
110100410,ACA6 ICU,26
110100019,ADULT EMERGENCY DEPARTMENT,101967
7101000,AMBULATORY SURGERY CENTER,214
7101002,ASC INTRA-OP,1769


### Diagnosis

In [13]:
# pat <-  paste0("^(", paste(code_list, collapse = "|"), ")")
dx %>% group_by(dx_name, icd10) %>% summarize(n=n()) %>% filter(str_detect(icd10, "^E11.")) %>% arrange(icd10)

`summarise()` has grouped output by 'dx_name'. You can override using the `.groups` argument.



dx_name,icd10,n
<chr>,<chr>,<int>
Diabetes mellitus with hyperosmolarity (CMS-HCC),E11.00,1
Diabetes mellitus with hyperosmolarity without hyperglycemic hyperosmolar nonketotic coma (CMS-HCC),E11.00,1
Diabetic hyperosmolar non-ketotic state (CMS-HCC),E11.00,20
Hyperosmolar non-ketotic state in patient with type 2 diabetes mellitus (CMS-HCC),E11.00,14
Type 2 diabetes mellitus with hyperosmolar nonketotic hyperglycemia (CMS-HCC),E11.00,7
Type 2 diabetes mellitus with hyperosmolarity without coma (CMS-HCC),E11.00,4
"Type 2 diabetes mellitus with hyperosmolarity without coma, unspecified long term insulin use status",E11.00,12
"Type 2 diabetes mellitus with hyperosmolarity without coma, unspecified whether long term insulin use (CMS-HCC)",E11.00,15
"Type 2 diabetes mellitus with hyperosmolarity without coma, without long-term current use of insulin (CMS-HCC)",E11.00,50


In [14]:
dx %>% group_by(dx_name, icd10) %>% filter(str_detect(dx_name, "ketoacidosis")) %>% summarize(n=n()) %>% arrange(-n)

`summarise()` has grouped output by 'dx_name'. You can override using the `.groups` argument.



dx_name,icd10,n
<chr>,<chr>,<int>
Type 1 diabetes mellitus with ketoacidosis without coma,E10.10,1268
Type 2 diabetes mellitus with ketoacidosis without coma,E11.10,872
Other specified diabetes mellitus with ketoacidosis without coma,E13.10,468
Diabetes mellitus due to underlying condition with ketoacidosis without coma,E08.10,433
Diabetic ketoacidosis without coma associated with type 1 diabetes mellitus (CMS-HCC),E10.10,231
Diabetic ketoacidosis without coma associated with type 2 diabetes mellitus (CMS-HCC),E11.10,165
Diabetic ketoacidosis without coma associated with diabetes mellitus due to underlying condition (CMS-HCC),E08.10,103
"Type II or unspecified type diabetes mellitus with ketoacidosis, not stated as uncontrolled",E11.10,82
"Type II or unspecified type diabetes mellitus with ketoacidosis, uncontrolled(250.12)",E13.10,81


### Treatment TEAM

In [3]:
team <- read_csv(file.path(datadir, "team_2021.csv"))
nrow(team) #2649157


── Column specification ────────────────────────────────────────────────────────
cols(
  anon_id = col_character(),
  pat_enc_csn_id_coded = col_double(),
  name = col_character(),
  prov_map_id = col_character(),
  treatment_start_time = col_datetime(format = ""),
  admit_time = col_datetime(format = "")
)




[1] 2649157

In [4]:
head(team, 3)

anon_id,pat_enc_csn_id_coded,name,prov_map_id,treatment_start_time,admit_time
<chr>,<dbl>,<chr>,<chr>,<dttm>,<dttm>
JC1000116,131066472308,Primary Team,SG1000039,2015-01-27 10:21:00,2015-01-28 00:46:00
JC1000116,131066472308,Case Manager,S1703,2015-01-28 01:41:00,2015-01-28 00:46:00
JC1000116,131066472308,Primary Team,SG1000037,2015-01-27 15:23:00,2015-01-28 00:46:00


In [5]:
team %>% group_by(name) %>% summarize(n=n()) %>% arrange(-n) #%>% filter(treatment_start_time < admit_time) 

name,n
<chr>,<int>
Registered Nurse,1123473
Respiratory Care Practitioner,279558
Case Manager,149380
Primary Team,122205
Nursing Assistant,119744
Social Worker,114648
Physical Therapist,92319
Occupational Therapist,84124
Clinical Dietitian,67437


### NOTE TYPE
- Remove ambulatory = 'Y' 
- Remove 'Letter', 'Telephone Encounter' --> ok to ignore note_status bc only Letter has NA note_status

In [6]:
note_type <- read_csv(file.path(datadir, "note_type_2021.csv"))
nrow(note_type) #14734359


── Column specification ────────────────────────────────────────────────────────
cols(
  anon_id = col_character(),
  pat_enc_csn_id_coded = col_double(),
  author_prov_map_id = col_character(),
  auth_lnked_prov_map_id = col_character(),
  note_status_c = col_double(),
  note_status = col_character(),
  note_type = col_character()
)




[1] 14734359

In [7]:
head(note_type, 2)

anon_id,pat_enc_csn_id_coded,author_prov_map_id,auth_lnked_prov_map_id,note_status_c,note_status,note_type
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
JC537660,9034168,SS0033777,SS0033777,2,Signed,X-Ray
JC883475,923137789,SS0099368,SS0099368,2,Signed,Telephone Encounter


In [8]:
# should remove "Letter and Telephone Encounter" and Ambulatory == Y
note_type %>% group_by(note_type) %>% summarize(n=n()) %>% arrange(-n) #

note_type,n
<chr>,<int>
Other Note,6034589
"Progress Note, Inpatient",2929296
ED Note,1459000
Progress/Discharge/Transfer Summary,1050731
Consultation Note,1039832
Letter,652060
"Progress Note, Outpatient",504331
Operative/Procedure Report,328563
Nursing Sign Out Note,289744


In [9]:
note_type %>% group_by(note_status, note_status_c) %>% summarize(n=n()) %>% arrange(-n) #

`summarise()` has grouped output by 'note_status'. You can override using the `.groups` argument.



note_status,note_status_c,n
<chr>,<dbl>,<int>
Signed,2,11957451
Addendum,3,2264991
NA,NA,499303
Cosign Needed,9,9892
Cosign Needed Addendum,11,2722


In [10]:
note_type %>% filter(note_type != "Letter") %>% group_by(note_status, note_status_c) %>% summarize(n=n()) %>% arrange(-n) #

`summarise()` has grouped output by 'note_status'. You can override using the `.groups` argument.



note_status,note_status_c,n
<chr>,<dbl>,<int>
Signed,2,11811535
Addendum,3,2258335
Cosign Needed,9,9723
Cosign Needed Addendum,11,2706


In [11]:
note_type %>% group_by(note_type, note_status, note_status_c) %>% summarize(n=n()) %>% arrange(-n) #

`summarise()` has grouped output by 'note_type', 'note_status'. You can override using the `.groups` argument.



note_type,note_status,note_status_c,n
<chr>,<chr>,<dbl>,<int>
Other Note,Signed,2,5411689
"Progress Note, Inpatient",Signed,2,2147356
ED Note,Signed,2,1379118
Progress/Discharge/Transfer Summary,Signed,2,1013017
"Progress Note, Inpatient",Addendum,3,778161
Other Note,Addendum,3,618151
Consultation Note,Signed,2,617311
"Progress Note, Outpatient",Signed,2,499899
Letter,NA,NA,499303


In [12]:
# all note_status == NA is from letters
note_type %>% filter(is.na(note_status_c)) %>% group_by(note_type, note_status, note_status_c) %>% summarize(n=n()) %>% arrange(-n) #

`summarise()` has grouped output by 'note_type', 'note_status'. You can override using the `.groups` argument.



note_type,note_status,note_status_c,n
<chr>,<chr>,<dbl>,<int>
Letter,NA,NA,499303


In [13]:
# should remove all letters
note_type %>% filter(note_type == "Letter") %>% group_by(note_type, note_status, note_status_c) %>% summarize(n=n()) %>% arrange(-n) #

`summarise()` has grouped output by 'note_type', 'note_status'. You can override using the `.groups` argument.



note_type,note_status,note_status_c,n
<chr>,<chr>,<dbl>,<int>
Letter,NA,NA,499303
Letter,Signed,2,145916
Letter,Addendum,3,6656
Letter,Cosign Needed,9,169
Letter,Cosign Needed Addendum,11,16


In [49]:
note_type <- note_type %>% filter(!note_type %in% c("Letter", "Telephone Encounter", "Progress Note, Outpatient")) 

In [50]:
# should remove "Letter and Telephone Encounter" and Ambulatory == Y, remove "Progress Note, Outpatient"
note_type %>% group_by(note_type) %>% summarize(n=n()) %>% arrange(-n) #

note_type,n
<chr>,<int>
Other Note,6034589
"Progress Note, Inpatient",2929296
ED Note,1459000
Progress/Discharge/Transfer Summary,1050731
Consultation Note,1039832
Operative/Procedure Report,328563
Nursing Sign Out Note,289744
History and Physical,274975
Discharge/Transfer Summary,153959


In [52]:
nrow(note_type)
sum(is.na(note_type$auth_lnked_prov_map_id))
sum(is.na(note_type$author_prov_map_id))

sum(note_type$auth_lnked_prov_map_id == note_type$author_prov_map_id)

[1] 13570312

[1] 711590

[1] 1601877

[1] NA

In [53]:
note_type %>% filter(is.na(auth_lnked_prov_map_id), !is.na(author_prov_map_id)) %>% head(3) # 220642

anon_id,pat_enc_csn_id_coded,author_prov_map_id,auth_lnked_prov_map_id,note_status_c,note_status,note_type,prov_map_id
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
JC1189563,25755855,SS0037854,NA,2,Signed,X-Ray,SS0037854
JC1000527,25682883,SS0037854,NA,2,Signed,X-Ray,SS0037854
JC649261,25438309,SS0037854,NA,2,Signed,X-Ray,SS0037854


In [54]:
note_type %>% filter(!is.na(auth_lnked_prov_map_id), is.na(author_prov_map_id)) %>% head(3) #1424750 

anon_id,pat_enc_csn_id_coded,author_prov_map_id,auth_lnked_prov_map_id,note_status_c,note_status,note_type,prov_map_id
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
JC704188,4100649,NA,SS0012098,2,Signed,X-Ray,SS0012098
JC687413,4337447,NA,SS0013328,2,Signed,X-Ray,SS0013328
JC598729,3481884,NA,SS0007279,2,Signed,X-Ray,SS0007279


In [55]:
note_type <- note_type %>% mutate(prov_map_id = ifelse(is.na(auth_lnked_prov_map_id), author_prov_map_id, auth_lnked_prov_map_id))
sum(is.na(note_type$prov_map_id))

[1] 492730

In [57]:
note_type %>% filter(is.na(prov_map_id)) %>% head(3)

anon_id,pat_enc_csn_id_coded,author_prov_map_id,auth_lnked_prov_map_id,note_status_c,note_status,note_type,prov_map_id
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
JC610754,300221548,NA,NA,2,Signed,ED Note,NA
JC2321743,1647030996,NA,NA,2,Signed,ED Note,NA
JC2847661,1482697869,NA,NA,2,Signed,ED Note,NA


In [58]:
head(team,1)
head(note_type,1)

anon_id,pat_enc_csn_id_coded,name,prov_map_id,treatment_start_time,admit_time
<chr>,<dbl>,<chr>,<chr>,<dttm>,<dttm>
JC1000116,131066472308,Primary Team,SG1000039,2015-01-27 10:21:00,2015-01-28 00:46:00


anon_id,pat_enc_csn_id_coded,author_prov_map_id,auth_lnked_prov_map_id,note_status_c,note_status,note_type,prov_map_id
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
JC537660,9034168,SS0033777,SS0033777,2,Signed,X-Ray,SS0033777


In [47]:
remove <- c('Nursing Assistant', 'Student Nurse', 'Medical Student', 'Relief Charge', 
            'ED Unit Secretary', 'Consulting Med Student', 'Primary Medical Student', 
            'Restorative Aide', 'ED Registrar', 'ED Tech', 'OB Tech')
rn <- c('Registered Nurse', 'Resource Nurse', 'Float Nurse', 'Triage Nurse', 'Licensed Vocational Nurse', 
         'Post-Transplant Nurse', 'Nursery Nurse', 'Delivery Nurse', 'Specialty Nurse')
md <- c('Primary Team', 'Emergency Resident', 'Primary Resident', 'Primary Intern', 'Senior Resident', 
        'Primary Advanced Practice Provider', 'Nurse Practitioner', 'Intern', 'Resident', 'Physician Assistant', 
         'Cardiologist', 'Primary Fellow', 'Pulmonologist', 'Fellow', 'Primary Sub-intern', 'Sub-Intern', 
         'Gastroenterologist', 'Infectious Disease', 'Chief Resident', 'Co-Attending', 'Primary Care Physician',  
         'Hematologist', 'Internist', 'Surgeon', 'Medical Oncologist', 'Dermatologist', 'Surgical Oncologist', 
         'Endocrinologist', 'Nephrologist', 'Trauma Attending', 'Diabetes MD', 'Trauma Resident', 
         'BMT Attending Provider', 'Urologist', 'Transplant Pulmonologist', 'Neurologist', 'Anesthesiologist', 
         'Primary Physician Assistant', 'Hepatologist', 'Transplant Surgeon', 'Obstetrician', 
         'Family Practitioner', 'Heart Failure Cardiologist', 'Radiation Oncologist', 'Referring Pulmonologist', 'Transplant Nephrologist',
         'Cross Cover Intern', 'Cross Cover Sub-Intern', 'Cross Cover Attending', 'Cross Cover Resident', 
         'Cross Cover Fellow', 'Cross Cover Advanced Practice Provider',
         'Consulting Service', 'Consulting Attending', 'Consulting Fellow', 'Consulting Resident', 'Consulting Intern', 
         'Psychologist', 'Consulting Medical Oncologist', 'Consulting Hematologist', 'Consulting Surgical Oncologist', 'Post-Transplant Nephrologist')

In [59]:
team_uni <- team %>% filter(!name %in% remove) %>% distinct(name, prov_map_id) 
note_team <- note_type %>% select(prov_map_id, note_type) %>% distinct() %>% left_join(team_uni)
nrow(note_team)

Joining, by = "prov_map_id"



[1] 111817

In [60]:
head(note_team, 3)

prov_map_id,note_type,name
<chr>,<chr>,<chr>
SS0033777,X-Ray,NA
SS0012098,X-Ray,NA
SS0013328,X-Ray,NA


In [61]:
note_team %>% group_by(name) %>% count() %>% arrange(-n)

name,n
<chr>,<int>
NA,48301
Registered Nurse,16505
Primary Team,5443
Emergency Resident,3921
Primary Resident,3555
Primary Intern,2878
Cross Cover Intern,2250
Resident,2236
Cross Cover Resident,2080


In [73]:
note_team %>% group_by(name, note_type) %>% count() %>% arrange(-n) %>% filter(n>10)

name,note_type,n
<chr>,<chr>,<int>
NA,Other Note,12933
NA,"Progress Note, Inpatient",10320
NA,Consultation Note,5962
NA,History and Physical,5161
Registered Nurse,Progress/Discharge/Transfer Summary,4886
NA,Operative/Procedure Report,4858
Registered Nurse,"Progress Note, Inpatient",4663
NA,Discharge/Transfer Summary,3978
Registered Nurse,Other Note,2690


In [72]:
note_team %>% group_by(name, note_type) %>% count() %>% arrange(name) %>% filter(n>10)

name,note_type,n
<chr>,<chr>,<int>
Additional Communicating Provider,Consultation Note,11
Additional Communicating Provider,Discharge/Transfer Summary,12
Additional Communicating Provider,History and Physical,13
Additional Communicating Provider,Operative/Procedure Report,13
Additional Communicating Provider,Other Note,14
Additional Communicating Provider,"Progress Note, Inpatient",15
Aging Adult Services Coordinator,Consultation Note,11
Aging Adult Services Coordinator,Other Note,11
Cardiologist,Consultation Note,13


In [71]:
note_team %>% group_by(note_type, name) %>% count() %>% arrange(note_type) %>% filter(n>10)

note_type,name,n
<chr>,<chr>,<int>
Consultation Note,Additional Communicating Provider,11
Consultation Note,Aging Adult Services Coordinator,11
Consultation Note,Cardiologist,13
Consultation Note,Case Manager,156
Consultation Note,Chief Resident,41
Consultation Note,Clinical Dietitian,62
Consultation Note,Clinical Pharmacist,16
Consultation Note,Co-Attending,81
Consultation Note,Consulting Attending,279
